#                    NLP assignment 2 
##                       2291588

# Define test sets (edit this one to put in new test set 4 and 5)


In [153]:
# Define test sets

testsets = ['twitter-test1.txt', 'twitter-test2.txt', 'twitter-test3.txt']
# testsets = ['twitter-test1.txt', 'twitter-test2.txt', 'twitter-test3.txt', 'twitter-test4.txt', 'twitter-test5.txt']
# I repeated two of test sets to make sure load more works. 

# Details of the code

## load libraries Run all of this part

In [154]:
# library preprocess
import re
import emoji
import json
from nltk.stem import WordNetLemmatizer
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
# from nltk.stem import PorterStemmer
from os.path import join


In [109]:
# import library Part 2
import pandas as pd
# bow tfidf ngram 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
#word2vec
import gensim
from gensim.models import Word2Vec
# normalize
from sklearn.preprocessing import normalize
from sklearn import preprocessing
# resampler
 # imbalanced data 
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
# for testing best resample method
from imblearn.under_sampling import TomekLinks
from imblearn.over_sampling import ADASYN
# hyper parameter
from sklearn.model_selection import  GridSearchCV
# bayesian
from sklearn.naive_bayes import MultinomialNB

# svc
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.metrics import classification_report
# random forest
from sklearn.ensemble import RandomForestClassifier
#MLP
from sklearn.neural_network import MLPClassifier
# ensemble
from sklearn.ensemble import VotingClassifier
from sklearn.calibration import CalibratedClassifierCV
import joblib
import pickle

In [155]:
# LSTM 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
# decay learning rate
import torch.optim.lr_scheduler as lr_scheduler
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from matplotlib import pyplot as plt
#get word index
import collections
# for attention
import torch.nn.functional as F
# for evaluation
from sklearn.metrics import f1_score
# for he initialization
import torch.nn.init as init

In [156]:
# First define a function to read data.
def read_twitter_data(data_path):
    list_name=[]
    file = open(data_path, 'r', encoding='utf-8')
    
    for line in file.readlines():
        dic = line
        list_name.append(dic)
    return list_name


In [157]:
# read train data
train_data=read_twitter_data("twitter-training-data.txt")
# read dev data
dev_data=read_twitter_data("twitter-dev-data.txt")
# read test 1 data
test_data1=read_twitter_data("twitter-test1.txt")
# read test 2 data
test_data2=read_twitter_data("twitter-test2.txt")
#read test 3 data
test_data3=read_twitter_data("twitter-test3.txt")


Abbreviation dictionary

In [158]:
#  abbreviation dictionary cited from Kaggle There are many better dictionaries but this one works well for sentiment analysis
# Twitter Sentiment Analysis with Naive Bayes 85%acc
# https://www.kaggle.com/code/lykin22/twitter-sentiment-analysis-with-naive-bayes-85-acc
abbreviations = {
    "$" : " dollar ",
    "€" : " euro ",
    "4ao" : "for adults only",
    "a.m" : "before midday",
    "a3" : "anytime anywhere anyplace",
    "aamof" : "as a matter of fact",
    "acct" : "account",
    "adih" : "another day in hell",
    "afaic" : "as far as i am concerned",
    "afaict" : "as far as i can tell",
    "afaik" : "as far as i know",
    "afair" : "as far as i remember",
    "afk" : "away from keyboard",
    "app" : "application",
    "approx" : "approximately",
    "apps" : "applications",
    "asap" : "as soon as possible",
    "asl" : "age, sex, location",
    "atk" : "at the keyboard",
    "ave." : "avenue",
    "aymm" : "are you my mother",
    "ayor" : "at your own risk", 
    "b&b" : "bed and breakfast",
    "b+b" : "bed and breakfast",
    "b.c" : "before christ",
    "b2b" : "business to business",
    "b2c" : "business to customer",
    "b4" : "before",
    "b4n" : "bye for now",
    "b@u" : "back at you",
    "bae" : "before anyone else",
    "bak" : "back at keyboard",
    "bbbg" : "bye bye be good",
    "bbc" : "british broadcasting corporation",
    "bbias" : "be back in a second",
    "bbl" : "be back later",
    "bbs" : "be back soon",
    "be4" : "before",
    "bfn" : "bye for now",
    "blvd" : "boulevard",
    "bout" : "about",
    "brb" : "be right back",
    "bros" : "brothers",
    "brt" : "be right there",
    "bsaaw" : "big smile and a wink",
    "btw" : "by the way",
    "bwl" : "bursting with laughter",
    "c/o" : "care of",
    "cet" : "central european time",
    "cf" : "compare",
    "cia" : "central intelligence agency",
    "csl" : "can not stop laughing",
    "cu" : "see you",
    "cul8r" : "see you later",
    "cv" : "curriculum vitae",
    "cwot" : "complete waste of time",
    "cya" : "see you",
    "cyt" : "see you tomorrow",
    "dae" : "does anyone else",
    "dbmib" : "do not bother me i am busy",
    "diy" : "do it yourself",
    "dm" : "direct message",
    "dwh" : "during work hours",
    "e123" : "easy as one two three",
    "eet" : "eastern european time",
    "eg" : "example",
    "embm" : "early morning business meeting",
    "encl" : "enclosed",
    "encl." : "enclosed",
    "etc" : "and so on",
    "faq" : "frequently asked questions",
    "fawc" : "for anyone who cares",
    "fb" : "facebook",
    "fc" : "fingers crossed",
    "fig" : "figure",
    "fimh" : "forever in my heart", 
    "ft." : "feet",
    "ft" : "featuring",
    "ftl" : "for the loss",
    "ftw" : "for the win",
    "fwiw" : "for what it is worth",
    "fyi" : "for your information",
    "g9" : "genius",
    "gahoy" : "get a hold of yourself",
    "gal" : "get a life",
    "gcse" : "general certificate of secondary education",
    "gfn" : "gone for now",
    "gg" : "good game",
    "gl" : "good luck",
    "glhf" : "good luck have fun",
    "gmt" : "greenwich mean time",
    "gmta" : "great minds think alike",
    "gn" : "good night",
    "g.o.a.t" : "greatest of all time",
    "goat" : "greatest of all time",
    "goi" : "get over it",
    "gps" : "global positioning system",
    "gr8" : "great",
    "gratz" : "congratulations",
    "gyal" : "girl",
    "h&c" : "hot and cold",
    "hp" : "horsepower",
    "hr" : "hour",
    "hrh" : "his royal highness",
    "ht" : "height",
    "ibrb" : "i will be right back",
    "ic" : "i see",
    "icq" : "i seek you",
    "icymi" : "in case you missed it",
    "idc" : "i do not care",
    "idgadf" : "i do not give a damn fuck",
    "idgaf" : "i do not give a fuck",
    "idk" : "i do not know",
    "ie" : "that is",
    "i.e" : "that is",
    "ifyp" : "i feel your pain",
    "IG" : "instagram",
    "iirc" : "if i remember correctly",
    "ilu" : "i love you",
    "ily" : "i love you",
    "imho" : "in my humble opinion",
    "imo" : "in my opinion",
    "imu" : "i miss you",
    "iow" : "in other words",
    "irl" : "in real life",
    "j4f" : "just for fun",
    "jic" : "just in case",
    "jk" : "just kidding",
    "jsyk" : "just so you know",
    "l8r" : "later",
    "lb" : "pound",
    "lbs" : "pounds",
    "ldr" : "long distance relationship",
    "lmao" : "laugh my ass off",
    "lmfao" : "laugh my fucking ass off",
    "lol" : "laughing out loud",
    "ltd" : "limited",
    "ltns" : "long time no see",
    "m8" : "mate",
    "mf" : "motherfucker",
    "mfs" : "motherfuckers",
    "mfw" : "my face when",
    "mofo" : "motherfucker",
    "mph" : "miles per hour",
    "mr" : "mister",
    "mrw" : "my reaction when",
    "ms" : "miss",
    "mte" : "my thoughts exactly",
    "nagi" : "not a good idea",
    "nbc" : "national broadcasting company",
    "nbd" : "not big deal",
    "nfs" : "not for sale",
    "ngl" : "not going to lie",
    "nhs" : "national health service",
    "nrn" : "no reply necessary",
    "nsfl" : "not safe for life",
    "nsfw" : "not safe for work",
    "nth" : "nice to have",
    "nvr" : "never",
    "nyc" : "new york city",
    "oc" : "original content",
    "og" : "original",
    "ohp" : "overhead projector",
    "oic" : "oh i see",
    "omdb" : "over my dead body",
    "omg" : "oh my god",
    "omw" : "on my way",
    "p.a" : "per annum",
    "p.m" : "after midday",
    "pm" : "prime minister",
    "poc" : "people of color",
    "pov" : "point of view",
    "pp" : "pages",
    "ppl" : "people",
    "prw" : "parents are watching",
    "ps" : "postscript",
    "pt" : "point",
    "ptb" : "please text back",
    "pto" : "please turn over",
    "qpsa" : "what happens", 
    "ratchet" : "rude",
    "rbtl" : "read between the lines",
    "rlrt" : "real life retweet", 
    "rofl" : "rolling on the floor laughing",
    "roflol" : "rolling on the floor laughing out loud",
    "rotflmao" : "rolling on the floor laughing my ass off",
    "rt" : "retweet",
    "ruok" : "are you ok",
    "sfw" : "safe for work",
     "sk8" : "skate",
    "smh" : "shake my head",
    "sq" : "square",
    "srsly" : "seriously", 
    "ssdd" : "same stuff different day",
    "tbh" : "to be honest",
    "tbs" : "tablespooful",
    "tbsp" : "tablespooful",
    "tfw" : "that feeling when",
    "thks" : "thank you",
    "tho" : "though",
    "thx" : "thank you",
    "tia" : "thanks in advance",
    "til" : "today i learned",
    "tl;dr" : "too long i did not read",
    "tldr" : "too long i did not read",
    "tmb" : "tweet me back",
    "tntl" : "trying not to laugh",
    "ttyl" : "talk to you later",
    "u" : "you",
    "u2" : "you too",
    "u4e" : "yours for ever",
    "utc" : "coordinated universal time",
    "w/" : "with",
    "w/o" : "without",
    "w8" : "wait",
    "wassup" : "what is up",
    "wb" : "welcome back",
    "wtf" : "what the fuck",
    "wtg" : "way to go",
    "wtpa" : "where the party at",
    "wuf" : "where are you from",
    "wuzup" : "what is up",
    "wywh" : "wish you were here",
    "yd" : "yard",
    "ygtr" : "you got that right",
    "ynk" : "you never know",
    "zzz" : "sleeping bored and tired"
}

## Define the preprocess function

In [159]:
def split_item(raw_data):
    split_list=[]
    for raw_data in raw_data:
        # Split the input string by '\t'
        split_str = raw_data.split('\t')
        split_list.append(split_str)
    return split_list
# cite from kaggle deal with abb
def convert_abb(tw):
    t=[]
    words=tw.split()
    t = [abbreviations[w.lower()] if w.lower() in abbreviations.keys() else w for w in words]
    return ' '.join(t)
def preprocess(text):
    # Common preprocess
    # delete stop_word but keep negation words
    stop_words = set(stopwords.words('english'))
    # negation_words = ['no', 'not']
    negation_words = ['no', 'not', "aren't", 'but',
                     'neither', 'nor', "ain't", "can't", "couldn't", "didn't", "doesn't", "hadn't", "hasn't", "haven't",
                     "isn't", "weren't", "won't", "wouldn't", "shouldn't", "mustn't"]
    stop_words = stop_words - set(negation_words)
    # remove URLs
    text = re.sub(r'http\S+', '', text)  
    # remove usernames
    text = re.sub('@[^\s]+', '', text)
    # remove digits
    text = re.sub(r'\d+', '', text) # remove digits
   
    # transfer abbrevations into words
    text=convert_abb(text)# translate abb
    # translate emoji
    text = emoji.demojize(text, delimiters=(" ", " ")) 
    #remove _ in emoji
    text = re.sub(r'_', r' ', text)
    # remove words with hastags
    text = re.sub(r'\s*#\S+', '', text)
    # remove punctuations and special characters
    text = re.sub(r'[^\w\s]', '', text) 
    # tokenize and convert to lowercase
    tokens = word_tokenize(text.lower())  
    # lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Remove words with length less than or equal to 2 ， as these word are not relevant to meaning except no
    lemmatized_tokens = [word for word in lemmatized_tokens if len(word) > 2 or word=='no' ]
    
    # Join tokens back into a string
    processed_twitter=text = ' '.join(lemmatized_tokens)
    return processed_twitter
 



## Part 2 Data scaling; deal with Imbalance data 
### Run all codes here

In [160]:
# split the data and make data ready for vectorization
def split_data(data1):
    X_train=[]
    Y_train=[]
    dic_X=[]
    for i in range(len(data1)):
        X_train.append(data1[i][2])
        Y_train.append(data1[i][1])
        dic_X.append(data1[i][0])
    return X_train,Y_train,dic_X
# we only need to preprocess the content 
def preprocess_data(input_data):
    dici=[]
    data1=split_item(input_data)
    for i in range(0,len(data1)):
        dici.append(preprocess(data1[i][2]))
    X,Y,D=split_data(data1)
    return dici,Y,D

    

In [161]:
# Now get id; sentiment and content for each data set
Xtrain_1,Ytrain,D=preprocess_data(train_data)
Xdev_1,Ydev,d0=preprocess_data(dev_data)
Xtest1_1,Ytest1,d1=preprocess_data(test_data1)
Xtest2_1,Ytest2,d2=preprocess_data(test_data2)
Xtest3_1,Ytest3,d3=preprocess_data(test_data3)


In [162]:
# other data
# Load training set, dev set and testing set
# I did not load train and dev data here as they have different use, test sets will only be used for test
data = {}
tweetids = {}
tweetgts = {}
tweets = {}
# Note the tweets[dataset] is preprocessed tweets
for dataset in  testsets:
    
    data[dataset] = read_twitter_data(dataset)
    Xt,Yt,Dt=preprocess_data(data[dataset])
    tweets[dataset]=Xt
    tweetids[dataset] = Dt
    tweetgts[dataset] = Yt

    # write code to read in the datasets here

## Test process 
code in this part is to test all features scaling and resampling method. It was turned to raw and no need to be run. Only change them to code if you want to test all features.

## Test step 1 Pick one of Four types of feature ( change raw to code for all step )
### Only Pick one of them!  no need to run if only want to see best model

### Bag of word 1

### Tfidf 2

### Ngram

### Word2Vec

### All out put data in this part are ready for putting in classifier Three approach has the same variable name for comvinence, so only run one of the three codes

### Test step 2 Four types of imbalanced data process + original data
 All data in this part are stored in list with 5 resample methods including original data Run all codes here

#### Changed to mark down as not used later use this to replace previous resampled data if you want to try all resample method 

###  Test step 3 pick one of Data scaling 
 All codes here are different kind of data scaling with the same variable name, please only run one of them.(after change to code)

### Normalized

### Max ABS

### Standarization

#### Min max

## Evaluation code given code must run this

In [163]:
# Skeleton: Evaluation code for the test sets
def read_test(testset):
    '''
    readin the testset and return a dictionary
    :param testset: str, the file name of the testset to compare
    '''
    id_gts = {}
    with open(testset, 'r', encoding='utf8') as fh:
        for line in fh:
            fields = line.split('\t')
            tweetid = fields[0]
            gt = fields[1]

            id_gts[tweetid] = gt

    return id_gts


def confusion(id_preds, testset, classifier):
    '''
    print the confusion matrix of {'positive', 'netative'} between preds and testset
    :param id_preds: a dictionary of predictions formated as {<tweetid>:<sentiment>, ... }
    :param testset: str, the file name of the testset to compare
    :classifier: str, the name of the classifier
    '''
    id_gts = read_test(testset)

    gts = []
    for m, c1 in id_gts.items():
        if c1 not in gts:
            gts.append(c1)

    gts = ['positive', 'negative', 'neutral']

    conf = {}
    for c1 in gts:
        conf[c1] = {}
        for c2 in gts:
            conf[c1][c2] = 0

    for tweetid, gt in id_gts.items():
        if tweetid in id_preds:
            pred = id_preds[tweetid]
        else:
            pred = 'neutral'
        conf[pred][gt] += 1

    print(''.ljust(12) + '  '.join(gts))

    for c1 in gts:
        print(c1.ljust(12), end='')
        for c2 in gts:
            if sum(conf[c1].values()) > 0:
                print('%.3f     ' % (conf[c1][c2] / float(sum(conf[c1].values()))), end='')
            else:
                print('0.000     ', end='')
        print('')

    print('')


def evaluate(id_preds, testset, classifier):
    '''
    print the macro-F1 score of {'positive', 'netative'} between preds and testset
    :param id_preds: a dictionary of predictions formated as {<tweetid>:<sentiment>, ... }
    :param testset: str, the file name of the testset to compare
    :classifier: str, the name of the classifier
    '''
    id_gts = read_test(testset)

    acc_by_class = {}
    for gt in ['positive', 'negative', 'neutral']:
        acc_by_class[gt] = {'tp': 0, 'fp': 0, 'tn': 0, 'fn': 0}

    catf1s = {}

    ok = 0
    for tweetid, gt in id_gts.items():
        if tweetid in id_preds:
            pred = id_preds[tweetid]
        else:
            pred = 'neutral'

        if gt == pred:
            ok += 1
            acc_by_class[gt]['tp'] += 1
        else:
            acc_by_class[gt]['fn'] += 1
            acc_by_class[pred]['fp'] += 1

    catcount = 0
    itemcount = 0
    macro = {'p': 0, 'r': 0, 'f1': 0}
    micro = {'p': 0, 'r': 0, 'f1': 0}
    semevalmacro = {'p': 0, 'r': 0, 'f1': 0}

    microtp = 0
    microfp = 0
    microtn = 0
    microfn = 0
    for cat, acc in acc_by_class.items():
        catcount += 1

        microtp += acc['tp']
        microfp += acc['fp']
        microtn += acc['tn']
        microfn += acc['fn']

        p = 0
        if (acc['tp'] + acc['fp']) > 0:
            p = float(acc['tp']) / (acc['tp'] + acc['fp'])

        r = 0
        if (acc['tp'] + acc['fn']) > 0:
            r = float(acc['tp']) / (acc['tp'] + acc['fn'])

        f1 = 0
        if (p + r) > 0:
            f1 = 2 * p * r / (p + r)

        catf1s[cat] = f1

        n = acc['tp'] + acc['fn']

        macro['p'] += p
        macro['r'] += r
        macro['f1'] += f1

        if cat in ['positive', 'negative']:
            semevalmacro['p'] += p
            semevalmacro['r'] += r
            semevalmacro['f1'] += f1

        itemcount += n

    micro['p'] = float(microtp) / float(microtp + microfp)
    micro['r'] = float(microtp) / float(microtp + microfn)
    micro['f1'] = 2 * float(micro['p']) * micro['r'] / float(micro['p'] + micro['r'])

    semevalmacrof1 = semevalmacro['f1'] / 2

    print(testset + ' (' + classifier + '): %.3f' % semevalmacrof1)

## Bayesian classifier

###  test step 4 run each classifier's test process

 Train bayesian to run change back to code

### best bayesian model

In [164]:
# get bayesian train data and test data preprocessed
# 1-3 Ngram
nb_vectorizer = CountVectorizer(ngram_range=(1, 3))
# change to ngram vector
X_train_bayesian = nb_vectorizer.fit_transform(Xtrain_1)
# resample 
ros = RandomOverSampler()
Xtrain_br, Ytrain_br = ros.fit_resample(X_train_bayesian, Ytrain)
Xtrainb = normalize(Xtrain_br, norm='l2')


In [165]:
# output for test data
# ngram features for test set
nb_ngram_feature={}
for datasets in testsets:
    nb_ngram_feature[datasets]=normalize(nb_vectorizer.transform(tweets[datasets]),norm='l2')
nb_classifier = MultinomialNB()
nb_classifier.fit(Xtrainb,Ytrain_br )
for item in testsets:
    dict_output={}
    
    y_predict=nb_classifier.predict(nb_ngram_feature[item])
    for i in range(0,len(y_predict)):
        dict_output[tweetids[item][i]]=y_predict[i]
    print('Results for Bayesian classifier on test set')
    evaluate(dict_output,item,'nb_classifier')
    confusion(dict_output,item,'nb_classifier')



Results for Bayesian classifier on test set
twitter-test1.txt (nb_classifier): 0.594
            positive  negative  neutral
positive    0.578     0.070     0.351     
negative    0.202     0.535     0.263     
neutral     0.208     0.133     0.659     

Results for Bayesian classifier on test set
twitter-test2.txt (nb_classifier): 0.622
            positive  negative  neutral
positive    0.658     0.059     0.283     
negative    0.159     0.534     0.307     
neutral     0.271     0.083     0.646     

Results for Bayesian classifier on test set
twitter-test3.txt (nb_classifier): 0.557
            positive  negative  neutral
positive    0.628     0.070     0.302     
negative    0.247     0.397     0.356     
neutral     0.261     0.119     0.620     



## SVM classifier


### Step 4 test

###  best svm model

In [166]:
# svm
# get svm train data and test data preprocessed
# 1-3 Ngram
svm_vectorizer = CountVectorizer(ngram_range=(1, 3))
# change to ngram vector
X_train_svm = svm_vectorizer.fit_transform(Xtrain_1)
# resample 
rus = RandomUnderSampler()
Xtrain_sr, Ytrain_sr = rus.fit_resample(X_train_svm, Ytrain)
Xtrains = normalize(Xtrain_sr, norm='l2')

# ngram features for test set
svm_ngram_feature={}
for datasets in testsets:
    svm_ngram_feature[datasets]=normalize(svm_vectorizer.transform(tweets[datasets]),norm='l2')



In [167]:

svm=LinearSVC()
svm.fit(Xtrains,Ytrain_sr)
for item in testsets:
    dict_output={}
    
    y_predict=svm.predict(svm_ngram_feature[item])
    for i in range(0,len(y_predict)):
        dict_output[tweetids[item][i]]=y_predict[i]
    print('Results for svm classifier on test set')
    evaluate(dict_output,item,'svm_classifier')
    confusion(dict_output,item,'svm_classifier')

Results for svm classifier on test set
twitter-test1.txt (svm_classifier): 0.632
            positive  negative  neutral
positive    0.630     0.053     0.317     
negative    0.192     0.545     0.264     
neutral     0.218     0.102     0.679     

Results for svm classifier on test set
twitter-test2.txt (svm_classifier): 0.662
            positive  negative  neutral
positive    0.702     0.039     0.259     
negative    0.190     0.523     0.287     
neutral     0.272     0.072     0.657     

Results for svm classifier on test set
twitter-test3.txt (svm_classifier): 0.583
            positive  negative  neutral
positive    0.676     0.056     0.268     
negative    0.196     0.422     0.381     
neutral     0.301     0.096     0.604     



## Random Forest


### step 4 test

###  Best RF model out put slow so saved. change back to code to run

In [133]:
# rf
# get rf train data and test data preprocessed
# 1-3 Ngram
rf_vectorizer = CountVectorizer()
# change to ngram vector
X_train_rf = rf_vectorizer.fit_transform(Xtrain_1)
# resample 
rus = RandomUnderSampler()
Xtrain_rr, Ytrain_rr = rus.fit_resample(X_train_rf, Ytrain)
Xtrainr = normalize(Xtrain_rr, norm='l2')

# ngram features for test set
rf_bow_feature={}
for datasets in testsets:
    rf_bow_feature[datasets]=normalize(rf_vectorizer.transform(tweets[datasets]),norm='l2')



In [ ]:


# Create a random forest classifier with 100 trees
rf = RandomForestClassifier(n_estimators=100)

# Fit the random forest classifier to the training data
rf.fit(Xtrainr, Ytrain_rr)

for item in testsets:
    dict_output={}
    
    y_predict=rf.predict(rf_bow_feature[item])
    for i in range(0,len(y_predict)):
        dict_output[tweetids[item][i]]=y_predict[i]
    print('Results for rf classifier on test set')
    evaluate(dict_output,item,'rf')
    confusion(dict_output,item,'rf')


In [ ]:
# save
filename = 'random_forest_model.sav'
pickle.dump(rf, open(filename, 'wb'))



In [134]:
filename = 'random_forest_model.sav'
# load the model from the file
rf_loaded = pickle.load(open(filename, 'rb'))
for item in testsets:
    dict_output={}
    
    y_predict=rf_loaded.predict(rf_bow_feature[item])
    for i in range(0,len(y_predict)):
        dict_output[tweetids[item][i]]=y_predict[i]
    print('Results for rf classifier on test set')
    evaluate(dict_output,item,'rf')
    confusion(dict_output,item,'rf')

Results for rf classifier on test set
twitter-test1.txt (rf): 0.590
            positive  negative  neutral
positive    0.643     0.059     0.298     
negative    0.239     0.522     0.239     
neutral     0.262     0.111     0.628     

Results for rf classifier on test set
twitter-test2.txt (rf): 0.603
            positive  negative  neutral
positive    0.712     0.053     0.235     
negative    0.208     0.480     0.312     
neutral     0.374     0.067     0.559     

Results for rf classifier on test set
twitter-test3.txt (rf): 0.557
            positive  negative  neutral
positive    0.683     0.069     0.248     
negative    0.255     0.428     0.317     
neutral     0.298     0.101     0.601     



## MLP ( deep learning) slow and low performance so not included

## Ensemble

In [135]:
# joblib.dump(svc, 'svc_model.joblib')
# The model trained by svc but not linear svc
# load model
svc_loaded = joblib.load('svc_model.joblib')

In [136]:
# change svc to probablity 
calibrated_svc = CalibratedClassifierCV(svm)
# The following ensemble is too slow so I saved model.

In [ ]:
ensemble_classifier = VotingClassifier(estimators=[('nb', nb_classifier),('svc1',svc_loaded),
                                                   ('rf', rf_loaded),('svc2',calibrated_svc)],
                                                   
                                        voting='soft')

# Train the ensemble classifier on your data
# under sampler ngram same as features in svc
ensemble_classifier.fit(Xtrains, Ytrain_sr)
# over sampler svm
#ensemble_classifier.fit(Xtrainb, Ytrain_br)

In [ ]:
# save the model
joblib.dump(ensemble_classifier, 'ensemble_classifier.pkl')



In [137]:
# load the trained ensemble classifier from disk
# the file is 600 mb but training requires few hours
ensemble_classifier = joblib.load('ensemble_classifier.pkl')

for item in testsets:
    dict_output={}
    
    y_predict=ensemble_classifier.predict(svm_ngram_feature[item])
    for i in range(0,len(y_predict)):
        dict_output[tweetids[item][i]]=y_predict[i]
    print('Results for ensemble_classifier classifier on test set')
    evaluate(dict_output,item,'ensemble_classifier')
    confusion(dict_output,item,'ensemble_classifier')

Results for ensemble_classifier classifier on test set
twitter-test1.txt (ensemble_classifier): 0.631
            positive  negative  neutral
positive    0.633     0.059     0.308     
negative    0.180     0.574     0.246     
neutral     0.211     0.110     0.679     

Results for ensemble_classifier classifier on test set
twitter-test2.txt (ensemble_classifier): 0.638
            positive  negative  neutral
positive    0.689     0.045     0.266     
negative    0.189     0.524     0.288     
neutral     0.315     0.079     0.605     

Results for ensemble_classifier classifier on test set
twitter-test3.txt (ensemble_classifier): 0.583
            positive  negative  neutral
positive    0.670     0.061     0.268     
negative    0.216     0.435     0.349     
neutral     0.285     0.097     0.617     



## Part 3 LSTM Glove

In [138]:
# Load the embeddings into a dictionary
embeddings_index = {}
with open( 'glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Define the maximum number of words 
max_words = 5000

# Define the embedding dimension
embedding_dim = 100

# Initialize an embedding matrix with zeros
embedding_matrix = np.zeros((max_words, embedding_dim))


In [140]:
# Split the text into individual words
words = []
for sentence in Xtrain_1:
    words += sentence.lower().split()
# Get the most common words and their counts
word_counts = collections.Counter(words)

# Select the top 5000 words by frequency
top_words = [word for word, count in word_counts.most_common(4999)]
# build word index
word_index = {}
for word in top_words:
    if word not in word_index:
        word_index[word] = len(word_index) + 1
word_index['<PAD>'] = 0


In [141]:
# I have one function called preprocess_data in first part , as this is second question , so I called it preprocess_data2
def preprocess_data2(x_train, y_train, word_index, max_len=20):
    word_index=word_index
   
    X_train_processed = []
    for sentence in x_train:
        x = [word_index.get(word, 0) for word in sentence.split()][:max_len]
        x += [0] * (max_len - len(x))
        X_train_processed.append(x)
    Y_train_processed = [0 if label == 'positive' else 1 if label == 'neutral' else 2 for label in y_train]
    return X_train_processed, Y_train_processed


# Train test preprocess requires balancing so different from others  Also, Shuffle= True 
batch_size=32
X_train_processed, Y_train_processed = preprocess_data2(Xtrain_1, Ytrain, word_index)
rus = RandomOverSampler()
X_train_balanced, Y_train_balanced = rus.fit_resample(X_train_processed, Y_train_processed)
# RandomUnderSampler()


# Load and transfer data into Pytorch Tensor
X_train_tensor = torch.tensor(X_train_balanced)
Y_train_tensor = torch.tensor(Y_train_balanced)

# build tensor data set
dataset = TensorDataset(X_train_tensor, Y_train_tensor)

# Create train set loader use shuffle =True

train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)








In [142]:
def loader_generater(xdata,ydata):
    b_size=32
    X_processed1, Y_processed1 = preprocess_data2(xdata, ydata, word_index)

    # transfer data in to pytorch tensor
    X_tensor0 = torch.tensor(X_processed1)
    Y_tensor0 = torch.tensor(Y_processed1)

    # Create 
    dataset0 = TensorDataset(X_tensor0, Y_tensor0)

    # Create data loader

    test_loader0 = DataLoader(dataset0, batch_size=b_size, shuffle=False)
    return test_loader0

In [143]:
# Fill in the embedding matrix with the GloVe vectors for the words in our reference word index
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # Words not found in the embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    

In [145]:
# Define list of sentences 97% word length are lower than 20, 56% are lower than 15 , so choose 20.
sentences = Xtrain_1

# Split each sentence into a list of words and count the number of words
num_words = [len(sentence.split()) for sentence in sentences]

# Count the number of words in specific ranges
count_15_plus = sum(1 for n in num_words if n < 15)/len(sentences)

count_15_to_20 = sum(1 for n in num_words if 15 <= n <= 20)/len(sentences)
count_20_plus = sum(1 for n in num_words if n > 20)/len(sentences)
# Print the counts
print("Number of sentences with 15 words:", count_15_plus)

print("Number of sentences with between 15 and 20 words:", count_15_to_20)
print("Number of sentences with more than 20 words:", count_20_plus)


Number of sentences with 15 words: 0.5659963193720761
Number of sentences with between 15 and 20 words: 0.40158754794793905
Number of sentences with more than 20 words: 0.03241613267998492


In [ ]:
### Three models only run one and fit the relu with attention one .

In [ ]:
#### This is the best model

In [146]:
# best model among 4 use this one and fit the saved model with this one

num_words=5000
embedding_dim=100

class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.embedding = nn.Embedding(num_words, embedding_dim)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        
        self.lstm = nn.LSTM(embedding_dim, 64, batch_first=True, bidirectional=True)
        self.attention = nn.Sequential(nn.Linear(128, 64),
                                       nn.Tanh(),
                                       nn.Linear(64, 1))
        self.dropout = nn.Dropout(p=0.2)
        self.fc = nn.Sequential(nn.Linear(128, 32),
                                nn.ReLU(),
                                nn.Dropout(p=0.5),
                                nn.Linear(32, 3),
                                nn.ReLU())
        
        # He initializer
        init.kaiming_normal_(self.lstm.weight_ih_l0)
        init.kaiming_normal_(self.lstm.weight_hh_l0)
        init.kaiming_normal_(self.lstm.weight_ih_l0_reverse)
        init.kaiming_normal_(self.lstm.weight_hh_l0_reverse)
        
    def forward(self, x):
        embeddings = self.embedding(x)
        h0 = init.kaiming_normal_(torch.zeros(2, len(x), 64), mode='fan_in', nonlinearity='relu')
        c0 = init.kaiming_normal_(torch.zeros(2, len(x), 64), mode='fan_in', nonlinearity='relu')
        output, (h0, c0) = self.lstm(embeddings, (h0, c0))
        
        # apply attention
        attn_weights = self.attention(output)
        attn_weights = F.softmax(attn_weights, dim=1)
        attn_output = torch.bmm(output.transpose(1, 2), attn_weights).squeeze(2)
        
        attn_output = self.dropout(attn_output)
        output = self.fc(attn_output)
        return output


In [147]:


# Load the model from a file
PATHa = 'lstm_01.pth'
model2 = MyModel()
model2.load_state_dict(torch.load(PATHa))

<All keys matched successfully>

In [148]:
# Evaluate model on train set
def predict_model(xt,yt,di,lstm_model):
    test_loader=loader_generater(xt,yt)
    X_p,Y_p=preprocess_data2(xt, yt, word_index)
    model1=lstm_model
    model1.eval()
    #change to test model for example drop rate will disappear.
    with torch.no_grad():
        correct = 0
        total = 0
        y_pred = []
        for data in test_loader:
            inputs, labels = data
            outputs = model1(inputs)
            predicted = torch.argmax(outputs, dim=1)
            
            y_pred.extend(predicted.tolist())   
            
            
            
            total += labels.size(0)
            correct += (predicted == labels).sum().item()


    y_pred = np.array(y_pred).flatten()
    y_pred_back = []

    for label in y_pred:
        if label == 0:
            y_pred_back.append('positive')
        elif label == 1:
            y_pred_back.append('neutral')
        else:
            y_pred_back.append('negative')
    accuracy = 100 * correct / total
    print('Accuracy on test set: %d %%' % accuracy)

    # Get predicted values
    
    

    # calculate F1 score I use this to compare with sem eval 2017 performance
    f1 = f1_score(Y_p, y_pred, average='macro')
    print('F1 score:', f1)

    
    dicti2={}
    for i in range(0,len(y_pred_back)):
        dicti2[di[i]]=y_pred_back[i]
    return dicti2


In [97]:
# change model2to model1 to see the new trained model.
lstm0=predict_model(Xtrain_1,Ytrain,D,model2)
evaluate(lstm0,'twitter-training-data.txt','lstm')
confusion(lstm0,'twitter-training-data.txt','lstm')

Accuracy on test set: 66 %
F1 score: 0.6696740339205461
twitter-training-data.txt (lstm): 0.705
            positive  negative  neutral
positive    0.693     0.018     0.289     
negative    0.072     0.539     0.389     
neutral     0.186     0.037     0.777     



In [98]:
# change model2 to model1 to see the new trained model.
lstmd=predict_model(Xdev_1,Ydev,d0,model2)
evaluate(lstmd,'twitter-dev-data.txt','lstm')
confusion(lstmd,'twitter-dev-data.txt','lstm')


Accuracy on test set: 61 %
F1 score: 0.6156307286101398
twitter-dev-data.txt (lstm): 0.647
            positive  negative  neutral
positive    0.648     0.037     0.316     
negative    0.109     0.486     0.405     
neutral     0.211     0.071     0.718     



In [99]:
# change model2 to model1 to see new trained model
lstm1=predict_model(Xtest1_1,Ytest1,d1,model2)
lstm2=predict_model(Xtest2_1,Ytest2,d2,model2)
lstm3=predict_model(Xtest3_1,Ytest3,d3,model2)
evaluate(lstm1,'twitter-test1.txt','lstm')
confusion(lstm1,'twitter-test1.txt','lstm')
evaluate(lstm2,'twitter-test2.txt','lstm')
confusion(lstm2,'twitter-test2.txt','lstm')
evaluate(lstm3,'twitter-test3.txt','lstm')
confusion(lstm3,'twitter-test3.txt','lstm')

Accuracy on test set: 64 %
F1 score: 0.6350923392676382
Accuracy on test set: 65 %
F1 score: 0.6183694889998699
Accuracy on test set: 59 %
F1 score: 0.5779451897510944
twitter-test1.txt (lstm): 0.641
            positive  negative  neutral
positive    0.675     0.043     0.281     
negative    0.197     0.517     0.287     
neutral     0.227     0.085     0.688     

twitter-test2.txt (lstm): 0.642
            positive  negative  neutral
positive    0.739     0.033     0.228     
negative    0.230     0.470     0.300     
neutral     0.339     0.058     0.603     

twitter-test3.txt (lstm): 0.585
            positive  negative  neutral
positive    0.704     0.058     0.238     
negative    0.199     0.392     0.409     
neutral     0.290     0.085     0.625     



# Part 4 the  final output function for all classifier(only best performance)

In [ ]:
# Buid traditional sentiment classifiers. An example classifier name 'svm' is given
# in the code below. You should replace the other two classifier names
# with your own choices. For features used for classifier training, 
# the 'bow' feature is given in the code. But you could also explore the 
# use of other features.
# I changed the structure of this part as i don't want to output all features even I tried them all
# I want to only output best model


In [149]:
# Best bayesian model
for item in testsets:
    dict_output={}
    
    y_predict=nb_classifier.predict(nb_ngram_feature[item])
    for i in range(0,len(y_predict)):
        dict_output[tweetids[item][i]]=y_predict[i]
    print('Results for Bayesian classifier on test set')
    evaluate(dict_output,item,'nb_classifier')
    confusion(dict_output,item,'nb_classifier')

Results for Bayesian classifier on test set
twitter-test1.txt (nb_classifier): 0.589
            positive  negative  neutral
positive    0.576     0.071     0.354     
negative    0.211     0.535     0.254     
neutral     0.202     0.140     0.658     

Results for Bayesian classifier on test set
twitter-test2.txt (nb_classifier): 0.622
            positive  negative  neutral
positive    0.659     0.057     0.284     
negative    0.168     0.513     0.319     
neutral     0.280     0.081     0.639     

Results for Bayesian classifier on test set
twitter-test3.txt (nb_classifier): 0.570
            positive  negative  neutral
positive    0.639     0.068     0.293     
negative    0.225     0.419     0.356     
neutral     0.254     0.120     0.626     



In [101]:
# Best svm model ( saved model) 
for item in testsets:
    dict_output={}
    
    y_predict=svm.predict(svm_ngram_feature[item])
    for i in range(0,len(y_predict)):
        dict_output[tweetids[item][i]]=y_predict[i]
    print('Results for svm classifier on test set')
    evaluate(dict_output,item,'svm_classifier')
    confusion(dict_output,item,'svm_classifier')

Results for svm classifier on test set
twitter-test1.txt (svm_classifier): 0.636
            positive  negative  neutral
positive    0.640     0.047     0.313     
negative    0.185     0.548     0.267     
neutral     0.217     0.109     0.674     

Results for svm classifier on test set
twitter-test2.txt (svm_classifier): 0.648
            positive  negative  neutral
positive    0.697     0.042     0.261     
negative    0.204     0.512     0.283     
neutral     0.315     0.064     0.621     

Results for svm classifier on test set
twitter-test3.txt (svm_classifier): 0.589
            positive  negative  neutral
positive    0.680     0.062     0.258     
negative    0.204     0.436     0.360     
neutral     0.287     0.090     0.623     

Results for svm classifier on test set
twitter-test4.txt (svm_classifier): 0.589
            positive  negative  neutral
positive    0.680     0.062     0.258     
negative    0.204     0.436     0.360     
neutral     0.287     0.090     0.623   

In [150]:
# Best random forest  saved model
# To generate new model Best RF model out put slow so saved. change back to code to run
filename = 'random_forest_model.sav'
# load the model from the file
rf_loaded = pickle.load(open(filename, 'rb'))
for item in testsets:
    dict_output={}
    
    y_predict=rf_loaded.predict(rf_bow_feature[item])
    for i in range(0,len(y_predict)):
        dict_output[tweetids[item][i]]=y_predict[i]
    print('Results for rf classifier on test set')
    evaluate(dict_output,item,'rf')
    confusion(dict_output,item,'rf')

Results for rf classifier on test set
twitter-test1.txt (rf): 0.590
            positive  negative  neutral
positive    0.643     0.059     0.298     
negative    0.239     0.522     0.239     
neutral     0.262     0.111     0.628     

Results for rf classifier on test set
twitter-test2.txt (rf): 0.603
            positive  negative  neutral
positive    0.712     0.053     0.235     
negative    0.208     0.480     0.312     
neutral     0.374     0.067     0.559     

Results for rf classifier on test set
twitter-test3.txt (rf): 0.557
            positive  negative  neutral
positive    0.683     0.069     0.248     
negative    0.255     0.428     0.317     
neutral     0.298     0.101     0.601     



In [151]:
# ensemble model saved model
# to run the model run all in Ensemble
# load the trained ensemble classifier from disk
ensemble_classifier = joblib.load('ensemble_classifier.pkl')

for item in testsets:
    dict_output={}
    
    y_predict=ensemble_classifier.predict(svm_ngram_feature[item])
    for i in range(0,len(y_predict)):
        dict_output[tweetids[item][i]]=y_predict[i]
    print('Results for ensemble_classifier classifier on test set')
    evaluate(dict_output,item,'ensemble_classifier')
    confusion(dict_output,item,'ensemble_classifier')

Results for ensemble_classifier classifier on test set
twitter-test1.txt (ensemble_classifier): 0.631
            positive  negative  neutral
positive    0.633     0.059     0.308     
negative    0.180     0.574     0.246     
neutral     0.211     0.110     0.679     

Results for ensemble_classifier classifier on test set
twitter-test2.txt (ensemble_classifier): 0.638
            positive  negative  neutral
positive    0.689     0.045     0.266     
negative    0.189     0.524     0.288     
neutral     0.315     0.079     0.605     

Results for ensemble_classifier classifier on test set
twitter-test3.txt (ensemble_classifier): 0.583
            positive  negative  neutral
positive    0.670     0.061     0.268     
negative    0.216     0.435     0.349     
neutral     0.285     0.097     0.617     



In [103]:
# Best lstm
# Load the model from a file change model2 to model1 for new trained model

# train model fit to check whether over fit
lstm0=predict_model(Xtrain_1,Ytrain,D,model2)
evaluate(lstm0,'twitter-training-data.txt','lstm')
confusion(lstm0,'twitter-training-data.txt','lstm')
# use all test set


Accuracy on test set: 67 %
F1 score: 0.6711897102223879
twitter-training-data.txt (lstm): 0.707
            positive  negative  neutral
positive    0.693     0.018     0.289     
negative    0.071     0.541     0.388     
neutral     0.186     0.037     0.777     



In [152]:
# LSTM model change model2 to model1 for new trained model
for item in testsets:
    y_predict={}
    y_predict=predict_model(tweets[item],tweetgts[item],tweetids[item],model2)
    print('Results for lstm classifier on test set')
    evaluate(y_predict,item,'lstm')
    confusion(y_predict,item,'lstm')

Accuracy on test set: 64 %
F1 score: 0.6366140959853323
Results for lstm classifier on test set
twitter-test1.txt (lstm): 0.642
            positive  negative  neutral
positive    0.681     0.043     0.275     
negative    0.196     0.509     0.295     
neutral     0.217     0.088     0.695     

Accuracy on test set: 66 %
F1 score: 0.6277978090958684
Results for lstm classifier on test set
twitter-test2.txt (lstm): 0.652
            positive  negative  neutral
positive    0.748     0.029     0.223     
negative    0.220     0.481     0.300     
neutral     0.330     0.060     0.610     

Accuracy on test set: 59 %
F1 score: 0.5778602053147516
Results for lstm classifier on test set
twitter-test3.txt (lstm): 0.583
            positive  negative  neutral
positive    0.706     0.059     0.235     
negative    0.204     0.389     0.407     
neutral     0.287     0.087     0.626     

